In [169]:
reference = open("mm10_refseq_genes_chr6_50M_60M.bed", "r")

In [170]:
reference

<_io.TextIOWrapper name='mm10_refseq_genes_chr6_50M_60M.bed' mode='r' encoding='UTF-8'>

In [171]:
Meth55 = open("E55Epi_SRR3083929_1.chr6_bismark_bt2_pe_sorted.bedGraph", "r")

In [172]:
Meth55

<_io.TextIOWrapper name='E55Epi_SRR3083929_1.chr6_bismark_bt2_pe_sorted.bedGraph' mode='r' encoding='UTF-8'>

In [173]:
Meth4 = open("E40ICM_SRR3083926_1.chr6_bismark_bt2_pe_sorted.bedGraph", "r")

In [174]:
Meth4

<_io.TextIOWrapper name='E40ICM_SRR3083926_1.chr6_bismark_bt2_pe_sorted.bedGraph' mode='r' encoding='UTF-8'>

In [175]:
ref_genes = []
for line in reference:
    info_line = []
    info = line.split('\t')
    info_line.append(info[12])
    info_line.append(info[4])
    info_line.append(info[5])
    ref_genes.append(info_line)

In [233]:
ref_genes[0:10]

[['Mpp6', '50110240', '50198598'],
 ['Mpp6', '50110240', '50198598'],
 ['Mpp6', '50110240', '50198598'],
 ['Dfna5', '50207402', '50261769'],
 ['Osbpl3', '50293326', '50382837'],
 ['Osbpl3', '50293326', '50456170'],
 ['Cycs', '50562562', '50566474'],
 ['5430402O13Rik', '50566642', '50594865'],
 ['5430402O13Rik', '50566642', '50594865'],
 ['Mir6371', '50572397', '50572508']]

In [177]:
Meth55_loc = []
i = 0
for line in Meth55:
    if i == 0:
        i += 1
    else: 
        info = line.split('\t')
        info_line = []
        info_line.append(info[1])
        info_line.append(float(info[3]))
        Meth55_loc.append(info_line)

In [234]:
Meth55_loc[0:10]

[['50000557', 0.0],
 ['50000626', 100.0],
 ['50000643', 100.0],
 ['50001247', 50.0],
 ['50001543', 0.0],
 ['50001663', 100.0],
 ['50001745', 100.0],
 ['50002769', 0.0],
 ['50002808', 100.0],
 ['50002809', 0.0]]

In [179]:
Meth4_loc = []
i = 0
for line in Meth4:
    if i == 0:
        i += 1
    else: 
        info = line.split('\t')
        info_line = []
        info_line.append(info[1])
        info_line.append(float(info[3]))
        Meth4_loc.append(info_line)

In [235]:
Meth4_loc[0:10]

[['49999926', 0.0],
 ['49999965', 0.0],
 ['50000074', 0.0],
 ['50000077', 0.0],
 ['50000556', 0.0],
 ['50000754', 0.0],
 ['50001246', 0.0],
 ['50001322', 0.0],
 ['50003676', 0.0],
 ['50003688', 0.0]]

In [181]:
#find methylation per gene
MethScore = []
i = 0
for line in Meth4_loc:
    if line[1] == 0:
        continue
    else:
        if int(line[0]) > int(ref_genes[i][1]) and int(line[0]) < int(ref_genes[i][2]):
            info = []
            info.append(ref_genes[i][0])
            info.append(line[1])
            MethScore.append(info)
        else:
            if int(line[0]) < int(ref_genes[i][1]):
                continue 
            elif int(line[0]) > int(ref_genes[i][2]):
                i += 1
                if i >= len(ref_genes):
                    break
                else:
                    info = []
                    info.append(ref_genes[i][0])
                    info.append(line[1])
                    MethScore.append(info)

In [236]:
MethScore[0:10]

[['Mpp6', 74.74834437086092],
 ['Mpp6', 100.0],
 ['Mpp6', 100.0],
 ['Dfna5', 84.11320754716981],
 ['Osbpl3', 82.56129032258065],
 ['Osbpl3', 81.27536231884058],
 ['Cycs', 49.5],
 ['5430402O13Rik', 83.74358974358974],
 ['5430402O13Rik', 100.0],
 ['Mir6371', 25.0]]

In [225]:
#find methylation per gene for E4
score = 0 #will sum the methylation %
MethScoreE4 = [] #will store all gene names and methylation scores for each gene
ScoreInfo = [] #will score in a single line a single gene name and its score so that its added to Methscore together 
site_num = 0  #will count the number of sites in a gene
i = 0 #will count the gene position so that we stay in the samne gene for the sites that are within that gene's range
for line in Meth4_loc: #for each line of the Meth4 data
    if line[1] == 0: #if methylation of that site is 0 skip it
        continue
    else: #if methylation of that site is not 0 do this
        if int(line[0]) > int(ref_genes[i][1]) and int(line[0]) < int(ref_genes[i][2]): #if the site is within a gene
            score += int(line[1]) #save the meth score
            site_num += 1 #count the number of sites 
        else: #if its not within the range of a gene
            if int(line[0]) < int(ref_genes[i][1]): #if its before a gene
                continue #skip it
            elif int(line[0]) > int(ref_genes[i][2]): #if its in the range of the next gene
                i += 1 #change the position of the gene 
                if i >= len(ref_genes): #if its the last gene
                    ScoreInfo = [] #clean last gene's info
                    ScoreInfo.append(ref_genes[i-1][0]) #add the gene's name
                    meth_score = score / site_num #calculate the total score
                    ScoreInfo.append(meth_score) #add the gene's meth score
                    MethScoreE4.append(ScoreInfo) #add the total info to the list of total genes
                    break #finish the loop
                else: #if it's a new gene remember the info on this new gene and save the info on the last gene
                    ScoreInfo = [] 
                    ScoreInfo.append(ref_genes[i-1][0])
                    meth_score = score / site_num
                    ScoreInfo.append(meth_score)
                    MethScoreE4.append(ScoreInfo)
                    score = 0
                    site_num = 0
                    score += int(line[1]) 
                    site_num += 1

In [237]:
MethScoreE4[0:10]

[['Mpp6', 74.74834437086092],
 ['Mpp6', 100.0],
 ['Mpp6', 100.0],
 ['Dfna5', 84.11320754716981],
 ['Osbpl3', 82.56129032258065],
 ['Osbpl3', 81.27536231884058],
 ['Cycs', 49.5],
 ['5430402O13Rik', 83.74358974358974],
 ['5430402O13Rik', 100.0],
 ['Mir6371', 25.0]]

In [231]:
#find methylation per gene for E5
score = 0 #will sum the methylation %
MethScoreE55 = [] #will store all gene names and methylation scores for each gene
ScoreInfo = [] #will score in a single line a single gene name and its score so that its added to Methscore together 
site_num = 0  #will count the number of sites in a gene
i = 0 #will count the gene position so that we stay in the samne gene for the sites that are within that gene's range
for line in Meth55_loc: #for each line of the Meth4 data
        if int(line[0]) > int(ref_genes[i][1]) and int(line[0]) < int(ref_genes[i][2]): #if the site is within a gene
            score += int(line[1]) #save the meth score
            site_num += 1 #count the number of sites 
        else: #if its not within the range of a gene
            if int(line[0]) < int(ref_genes[i][1]): #if its before a gene
                continue #skip it
            elif int(line[0]) > int(ref_genes[i][2]): #if its in the range of the next gene
                i += 1 #change the position of the gene 
                if i >= len(ref_genes): #if its the last gene
                    ScoreInfo = [] #clean last gene's info
                    ScoreInfo.append(ref_genes[i-1][0]) #add the gene's name
                    meth_score = score / site_num #calculate the total score
                    ScoreInfo.append(meth_score) #add the gene's meth score
                    MethScoreE55.append(ScoreInfo) #add the total info to the list of total genes
                    break #finish the loop
                else: #if it's a new gene remember the info on this new gene and save the info on the last gene
                    ScoreInfo = [] 
                    ScoreInfo.append(ref_genes[i-1][0])
                    meth_score = score / site_num
                    ScoreInfo.append(meth_score)
                    MethScoreE55.append(ScoreInfo)
                    score = 0
                    site_num = 0
                    score += int(line[1]) 
                    site_num += 1

In [238]:
MethScoreE55[0:10]

[['Mpp6', 71.58937198067633],
 ['Mpp6', 100.0],
 ['Mpp6', 100.0],
 ['Dfna5', 65.27098321342926],
 ['Osbpl3', 76.26102502979738],
 ['Osbpl3', 73.8079365079365],
 ['Cycs', 69.6842105263158],
 ['5430402O13Rik', 65.75545851528385],
 ['5430402O13Rik', 0.0],
 ['Mir6371', 0.0]]

In [240]:
i = 0
FoldChange_Total = []
for score in MethScoreE55:
    info = []
    fold_change = score[1]/MethScoreE4[i][1]
    info.append(score[0])
    info.append(fold_change)
    FoldChange_Total.append(info)
    i += 1

In [253]:
len(FoldChange_Total)

152

In [254]:
FoldChange_Total[0:15]

[['Mpp6', 0.9577385637531787],
 ['Mpp6', 1.0],
 ['Mpp6', 1.0],
 ['Dfna5', 0.7759897062161846],
 ['Osbpl3', 0.9236898397764003],
 ['Osbpl3', 0.9081219006860947],
 ['Cycs', 1.4077618288144604],
 ['5430402O13Rik', 0.7851999026626058],
 ['5430402O13Rik', 0.0],
 ['Mir6371', 0.0],
 ['4921507P07Rik', 0.8032876712328767],
 ['Npvf', 0.6765000000000001],
 ['C530044C16Rik', 0.7664316662994786],
 ['Mir148a', 0.6666666666666666],
 ['Gm6559', 0.6075843285145611]]

In [255]:
with open('FoldChange.txt', 'w') as FoldChange:
    for item in FoldChange_Total:
        FoldChange.write('%s\n' % item)